In [ ]:
import pandas as pd 
from bs4 import BeautifulSoup
import requests
import datetime
df=pd.read_csv('Data/results.csv')
df['DateTime']=df['Date']+' '+df['Time']
df['DateTime']=pd.to_datetime(df['DateTime'],format='%Y-%m-%d %H:%M:%S.%f',utc=True)
df['Date']=pd.to_datetime(df['Date'],format='%Y-%m-%d',utc=True)
df['DateTime']=df['DateTime'].dt.tz_convert('Europe/London')
lst4=[]
for date1 in df['DateTime'].dt.date.unique():
    url='https://library-calendars.ucl.ac.uk/widget/hours/grid?iid=4014&lid=0&date='+str(date1)
    soup=BeautifulSoup(requests.get(url).text,'html.parser')
    lib=['Great Ormond Street Institute of Child Health Library',
    'UCL East Library', 'Language & Speech Science Library', 'SSEES Library',
    'School of Pharmacy Library','Joint Library of Ophthalmology',
    'Bartlett Library', 'Graduate Hub',
    'Queen Square Library',
    'Cruciform Hub','Institute of Archaeology Library','   Science Library',
    'Main Library','Student Centre', 'IOE Library']
    x=soup.find_all('tr',{'class':'s-lc-whw-loc'})
    lst3=[('12:01am','11:59pm',a[1], date1) if 'Student Centre' in a[1] else (a[0][0],a[0][2],a[1].strip(),date1) if 'Closed' not in a[0] else ('Closed','Closed',a[1],date1) for a in [(tuple(i.find_all('td')[date1.weekday()+1].span.text.strip().split()),b) for b in set([l1 for l1 in lib for i in x if l1 in i.text]) for i in x if b in i.text]]
    lst3=[a if 'Closed' in a[0] else (datetime.datetime.strptime(a[0], "%I:%M%p") if ":" in a[0] else datetime.datetime.strptime(a[0], "%I%p"),datetime.datetime.strptime(a[1], "%I:%M%p") if ":" in a[1] else datetime.datetime.strptime(a[1], "%I%p"),str(a[2]),a[3]) for a in lst3]
    lst4.extend(lst3)
df1=pd.DataFrame(lst4,columns=['Opening Time', 'Closing Time','Library','Date'])
df1['Date']=pd.to_datetime(df1['Date'],format='%Y-%m-%d',utc=True)
df1=pd.merge(df,df1,on=['Date'],suffixes=['','_y'])
df1['isin']=df1.apply(lambda row: row['Library_y'] in row['Library'] and not ('Language' in row['Library'] and 'Language' not in row['Library_y']), axis=1)
df1=df1[df1['isin']==True]
df1=df1.drop(columns=['isin','Library_y','DateTime'])
df2=pd.read_csv('Data/resultsophr.csv')
df2=pd.concat([df1,df2],ignore_index=True)
df2.copy().to_csv('data/resultsmerged.csv', mode='a', header=True, index=False)